# 태스크 3: RAG 평가(RAGAS) 프레임워크를 사용하여 Amazon Bedrock 지식 기반으로 Q&A 애플리케이션을 구축하고 평가

이 태스크에서는 LangChain의 **AmazonKnowledgeBasesRetriever** 클래스, 체인, RAGAS 프레임워크를 사용하여 응답 평가를 위한 Q&A 애플리케이션을 구축하고 평가합니다. 여기에서는 지식 기반을 쿼리하여 유사성 검색을 기반으로 원하는 수의 문서 청크를 가져옵니다. 다음으로, 쿼리와 함께 문서 청크를 컨텍스트로 제공하여 텍스트 생성 LLM에 프롬프트를 입력합니다. 그런 다음 faithfulness, answer_relevancy, context_recall, context_precision, context_entity_recall, answer_similarity, answer_correctness, harmfulness, maliciousness, coherence, correctness, conciseness 같은 평가 지표를 사용하여 응답을 평가합니다.

<i aria-hidden="true" class="fas fa-info-circle" style="color:#007FAA"></i> **자세히 알아보기:** Ragas 프레임워크에서 사용되는 다양한 지표에 대한 추가 정보는 **[지표](https://docs.ragas.io/en/latest/concepts/metrics/index.html)** 를 참조하십시오.

### 패턴

검색 증강 생성(RAG) 패턴을 사용하여 솔루션을 구현합니다. RAG는 언어 모델 외부에서 데이터를 검색하고, 검색된 관련 데이터를 컨텍스트에 추가하여 프롬프트를 증강합니다. 이 태스크에서는 실습 프로비저닝 과정에서 기존에 생성된 지식 기반을 사용하여 쿼리에 대한 응답을 생성합니다.

#### 평가

- RAGAS를 활용하여 다음과 같은 지표를 평가합니다.
  - **충실도:** 주어진 컨텍스트에 대해 생성된 답변의 사실적 일관성을 측정합니다. 이 지표는 응답 및 검색된 컨텍스트를 통해 계산됩니다. 답은 (0,1) 범위로 조정됩니다. 숫자가 클수록 좋습니다.
  - **답변 관련성:** 이 지표는 생성된 답변이 주어진 프롬프트와 얼마나 관련이 있는지 평가하는 데 중점을 둡니다. 불완전하거나 중복된 정보가 포함된 답변에는 낮은 점수가 부여되며 점수가 높을수록 관련성이 더 높습니다. 이 지표는 질문, 컨텍스트, 답변을 사용하여 계산됩니다. 실제로 점수는 대부분 0~1 범위지만, 이러한 점수 범위가 수학적으로 보장되는 것은 아닙니다. 그 이유는 코사인 유사성 범위가 -1~1이기 때문입니다.
  - **컨텍스트 정밀도:** 컨텍스트 내에 존재하는 모든 실측 정보 관련 항목의 순위가 더 높은지 여부를 평가하는 지표입니다. 이상적으로는 모든 관련 청크가 최상위 순위에 있어야 합니다. 이 지표는 질문, ground_truth, 컨텍스트를 사용하여 계산됩니다. 값은 0~1 범위이고, 점수가 높을수록 정밀도가 더 높습니다.
  - **컨텍스트 재현율:** 이 지표는 검색된 컨텍스트가 주석이 달린 답변, 즉 Ground Truth로 간주되는 답변과 일치하는 정도를 측정합니다. 이 지표는 Ground Truth 및 검색된 컨텍스트를 기준으로 계산됩니다. 값 범위는 0~1이며 값이 높을수록 성능이 더 뛰어납니다.
  - **컨텍스트 엔터티 재현율:** 이 지표는 ground_truths에만 존재하는 엔터티의 수에 비례하여 ground_truths와 컨텍스트 양쪽 모두에 존재하는 엔터티의 수를 기준으로, 검색된 컨텍스트의 재현율을 측정합니다. 간단히 말해, ground_truths에서 재현되는 엔터티의 비율을 측정한 것입니다. 이 지표는 관광 안내 데스크, 역사 QA 등과 같은 사실 기반 사용 사례에 유용합니다. 이 지표는 ground_truths에 존재하는 엔터티와의 비교를 기준으로 엔터티의 검색 메커니즘을 평가하는 데 도움이 될 수 있습니다. 그 이유는 엔터티가 중요한 경우, 이러한 엔터티를 다루는 컨텍스트가 필요하기 때문입니다.
  - **답변 의미 유사성:** 답변 의미 유사성의 개념은 생성된 답변과 Ground Truth 간의 의미 유사성을 평가하는 것과 관련이 있습니다. 이 평가는 Ground Truth와 답변을 기준으로 하며, 값은 0~1 범위에 속합니다. 점수가 높을수록 생성된 답변과 Ground Truth가 더욱 잘 일치한다는 의미입니다.
  - **답변 정확성:** 답변 정확성 평가 시에는 생성된 답변의 정확도를 Ground Truth와 비교하여 측정하는 작업이 이루어집니다. 이 평가는 Ground Truth와 답변을 토대로 하며, 점수 범위는 0~1입니다. 점수가 높을수록 생성된 답변과 Ground Truth가 더 밀접하게 일치한다는 걸 나타내므로, 정확성이 더 높음을 뜻합니다. 답변의 정확성에는 2가지 중요한 측면, 즉 생성된 답변과 Ground Truth 간의 의미적 유사성과 사실적 유사성이 포함됩니다. 이러한 측면은 가중치 체계를 사용해 조합하여 답변 정확성 점수 공식을 만듭니다. 또한 사용자는 원하는 경우 '임계치' 값을 사용하여 결과 점수를 이진수로 반올림할 수도 있습니다.
  - **측면 비평:** 무해성 및 정확성 같은 사전 정의된 측면을 기준으로 제출물을 평가하기 위해 고안된 지표입니다. 측면 비평의 출력은 이진수이며, 이는 제출물이 정의된 측면과 일치하는지 여부를 나타냅니다. 이 평가는 '답변'을 입력으로 사용하여 실행됩니다.

이 태스크에서는 AnyCompany의 financial 10k 보고서(합성하여 생성된 데이터세트)를 텍스트 말뭉치로 사용하여 Q&A를 수행합니다. 이 데이터는 기존에 Amazon Bedrock의 지식 기반에 수집되었습니다.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 특정 사용 사례에서는 다양한 도메인 주제에 대해 서로 다른 파일을 동기화한 후 동일한 방식으로 이 노트북을 쿼리하여 지식 기반에서 Retrieve API를 사용해 모델 응답을 평가할 수 있습니다.

<i aria-hidden="true" class="fas fa-exclamation-circle" style="color:#7C5AED"></i> **주의:** **Run** 메뉴에서 **Run All Cells** 옵션을 사용하는 것보다 각 코드 셀을 개별적으로 실행하는 것이 좋습니다. 모든 셀을 한꺼번에 같이 실행하면 커널 충돌 또는 재시작 같은 예기치 않은 동작이 발생할 수 있습니다. 셀을 하나씩 실행하면 실행 흐름을 더욱 잘 제어하고, 잠재적인 오류를 조기에 발견하며, 의도한 대로 코드가 실행되도록 할 수 있습니다.

## 태스크 3.1: 환경 설정

이 노트북을 실행하려면 종속성, LangChain 및 RAGAS를 설치하고 업데이트된 boto3, botocore 패키지를 설치해야 합니다.

아래에 제공된 단계에 따라 필요한 패키지를 설정하십시오.

- 파운데이션 모델을 간접적으로 호출하기 위한 **bedrock-runtime** 생성에 필요한 라이브러리 가져오기
- LangChain 관련 라이브러리 가져오기
- Bedrock 모델 **amazon.titan-text-premier-v1:0**을 대규모 언어 모델로 초기화하여 RAG 패턴을 사용해 쿼리 완료
- Bedrock 모델 **amazon.nova-lite-v1:0**을 대규모 언어 모델로 초기화하여 RAG 평가 수행
- Bedrock 모델 **amazon.titan-embed-text-v 2:0**을 대규모 언어 임베딩 모델로 초기화하여 RAG 평가를 위한 임베딩 생성. 이는 지식 기반을 생성하는 데 사용된 것과 동일한 임베딩 모델입니다.
- 지식 기반과 통합된 LangChain 검색기 초기화
- 나중에 노트북에서 LLM과 검색기를 Q&A 애플리케이션 구축을 위한 체인으로 래핑하기

1. 다음의 코드 셀을 실행하여 Amazon Bedrock에 있는 기존 지식 기반의 ID를 확인합니다.

In [1]:
import botocore
import boto3

session = boto3.Session()
bedrock_client = session.client('bedrock-agent')

try:
    response = bedrock_client.list_knowledge_bases(
        maxResults=1  # We only need to retrieve the first Knowledge Base
    )
    knowledge_base_summaries = response.get('knowledgeBaseSummaries', [])

    if knowledge_base_summaries:
        kb_id = knowledge_base_summaries[0]['knowledgeBaseId']
        print(f"Knowledge Base ID: {kb_id}")
    else:
        print("No Knowledge Base summaries found.")
        
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

Knowledge Base ID: YLDEIZDHQG


2. 다음의 코드 셀을 실행하여 종속성을 설치합니다.

In [2]:
import boto3
import pprint

from langchain_aws import ChatBedrock
from langchain_aws import BedrockEmbeddings
from langchain_community.retrievers import AmazonKnowledgeBasesRetriever

pp = pprint.PrettyPrinter(indent=2)

bedrock_client = boto3.client('bedrock-runtime')

llm_for_text_generation = ChatBedrock(model_id="amazon.nova-lite-v1:0", client=bedrock_client)
llm_for_evaluation = ChatBedrock(model_id="amazon.nova-lite-v1:0", client=bedrock_client)

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0",client=bedrock_client)

## 태스크 3.2: LangChain에서 *AmazonKnowledgeBasesRetriever* 객체 생성

이 태스크에서는 LangChain에서 **AmazonKnowledgeBasesRetriever** 객체를 생성하여 지식 기반을 검색하고 관련 결과를 반환하므로, 시맨틱 검색 결과를 기반으로 사용자 지정 워크플로를 구축할 수 있는 제어 기능이 향상됩니다.

3. 다음의 코드 셀을 실행하여 **AmazonKnowledgeBasesRetriever** 객체를 생성합니다.

In [3]:
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 5}},
        # endpoint_url=endpoint_url,
        # region_name="us-east-1",
        # credentials_profile_name="<profile_name>",
    )

/tmp/ipykernel_374/3249585155.py:1: LangChainDeprecationWarning: The class `AmazonKnowledgeBasesRetriever` was deprecated in LangChain 0.3.16 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import AmazonKnowledgeBasesRetriever``.
  retriever = AmazonKnowledgeBasesRetriever(


## 태스크 3.3: RetrievalQA 체인을 사용하여 모델 간접 호출 및 응답 생성 

이 태스크에서는 다음과 같은 정보를 사용하여 모델을 간접적으로 호출하고 응답을 시각화합니다.

질문 =

```
Provide a list of few risks for AnyCompany financial in numbered list without description."
```

Ground Truth 답변 = 

```
1. Commodity Prices
2. Foreign Exchange Rates 
3. Equity Prices
4. Credit Risk
5. Liquidity Risk
...
...
```

4. 다음의 코드 셀을 실행하여 컨텍스트와 질문이 변수로 포함된 프롬프트를 생성합니다.

In [4]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""
prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context","question"])

5. 다음의 코드 셀을 실행해 사전 정의된 쿼리를 사용하여 모델을 간접적으로 호출하고 결과를 인쇄합니다.

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs): #concatenate the text from the page_content field in the output from retriever.invoke
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm_for_text_generation
    | StrOutputParser()
)

query = "Provide a list of ten risks for AnyCompany financial as a numbered list. Do not include descriptions."

response=chain.invoke(query)
print(response)

1. Market Risks
2. Commodity Prices
3. Foreign Exchange Rates
4. Equity Prices
5. Operational Risk
6. Regulatory Risk
7. Strategic Risk
8. Reputation Risk
9. Legal Risk
10. Environmental, Social, and Governance (ESG) Risk


## 태스크 3.4: 평가 데이터 준비

RAGAS는 참조가 없는 평가 프레임워크를 목표로 하므로, 평가 데이터세트에 필요한 준비가 최소화됩니다. 이 태스크에서는 아래와 같이 추론을 통해 나머지 정보를 준비할 수 있는 **question**과 **ground_truths** 페어를 준비합니다.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** **context_recall** 지표에 관심이 없는 경우, **ground_truths** 정보를 제공하지 않아도 됩니다. 이 태스크에서는 **questions**만 준비하면 됩니다.

5. 다음의 코드 셀을 실행하여 평가를 위해 *question*과 *ground_truths* 페어를 준비합니다. 제한이 발생하는 경우 재시도를 하면서 몇 분 동안 실행될 수 있습니다.

In [6]:
from datasets import Dataset
import time
import random

# Define questions and ground truths for RAGAS evaluation
questions = [
    "What was the primary reason for the increase in net cash by operating activities for AnyCompany Financial in 2021?",
    "Which year did AnyCompany Financial have the highest net cash used in investing activities, what was the primary reason?",
    "What was the primary source of cash inflows from financing activities for AnyCompany Financial in 2021?",
    "Calculate the year-over-year percentage change in cash and cash equivalents for AnyCompany Financial from 2020 to 2021.",
    "With the information provided, what can you infer about AnyCompany Financial's overall financial health and growth prospects?"
]

ground_truth = [
    "An increase in net cash provided by operating activities was primarily due to increases in net income and favorable changes in operating assets and liabilities.",
    "AnyCompany Financial had the highest net cash used in investing activities in 2021, at $360 million, compared to $290 million in 2020 and $240 million in 2019. The primary reason was an increase in purchases of property, plant, and equipment and marketable securities.",
    "The primary source of cash inflows from financing activities for AnyCompany Financial in 2021 was an increase in proceeds from the issuance of common stock and long-term debt.",
    "To calculate the year-over-year percentage change in cash and cash equivalents from 2020 to 2021: \
    2020 cash and cash equivalents: $350 million \
    2021 cash and cash equivalents: $480 million \
    Percentage change = (2021 value - 2020 value) / 2020 value * 100 \
    = ($480 million - $350 million) / $350 million * 100 \
    = 37.14% increase",
    "Based on information provided, AnyCompany Financial appears to be in a healthy financial position with good growth prospects. The company increased its net cash provided by operating activities, indicating strong profitability and efficient management of working capital. AnyCompany Financial has been investing in long-term assets, such as property, plant, and equipment, and marketable securities, which suggests plans for future growth and expansion. The company was able to finance its growth through the issuance of common stock and long-term debt, indicating confidence from investors and lenders. Overall, AnyCompany Financial's steady increase in cash and cash equivalents over the past three years provides a strong foundation for future growth and investment opportunities."
]

def get_model_response(query, chain, retriever, max_retries=5, wait_time=15):
    """Get response from the model with fixed wait time between retries"""
    for attempt in range(max_retries):
        try:
            # Configure Nova Lite with increased tokens
            nova_config = {
                "schemaVersion": "messages-v1",
                "messages": [{
                    "role": "user",
                    "content": [{"text": query}]
                }],
                "inferenceConfig": {
                    "maxTokens": 2048,
                    "temperature": 0.5,
                    "topP": 0.9,
                    "topK": 20
                }
            }
            
            # Try to invoke with config override
            try:
                answer = chain.invoke(
                    query,
                    config_override={"model_kwargs": nova_config}
                )
            except AttributeError:
                # If config_override doesn't work, try direct invocation
                answer = chain.invoke(query)
            
            context = [docs.page_content for docs in retriever.invoke(query)]
            print(f"Successfully processed query on attempt {attempt + 1}")
            return answer, context
            
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"Failed after {max_retries} attempts for query: {query[:50]}...")
                print(f"Error: {str(e)}")
                return None, None
            print(f"Attempt {attempt + 1} failed, waiting {wait_time} seconds before retry...")
            time.sleep(wait_time)

# Process questions one at a time with fixed delay
answers = []
contexts = []

print("Starting to process questions...")
for i, query in enumerate(questions, 1):
    print(f"\nProcessing question {i}/{len(questions)}")
    print(f"Query: {query[:100]}...")
    
    answer, context = get_model_response(query, chain, retriever)
    if answer is not None:
        answers.append(answer)
        contexts.append(context)
        print(f"Successfully processed question {i}")
    else:
        print(f"Failed to process question {i}")
    
    #if i < len(questions):
    #    print(f"Waiting 60 seconds before next question...")
    #    time.sleep(60)

# Create dataset for RAGAS evaluation
data = {
    "question": questions[:len(answers)],
    "ground_truth": ground_truth[:len(answers)],
    "answer": answers,
    "contexts": contexts
}

# Convert to dataset
dataset = Dataset.from_dict(data)

# Print dataset information
print("\nDataset Creation Summary:")
print(f"Total questions processed: {len(dataset)} out of {len(questions)}")
print(f"Columns available: {dataset.column_names}")

# Print sample entry
if len(dataset) > 0:
    print("\nSample Entry (First Question):")
    print(f"Question: {dataset[0]['question']}")
    print(f"Ground Truth: {dataset[0]['ground_truth']}")
    print(f"Model Answer: {dataset[0]['answer']}")
else:
    print("\nNo entries were successfully processed into the dataset.")


Starting to process questions...

Processing question 1/5
Query: What was the primary reason for the increase in net cash by operating activities for AnyCompany Fina...
Successfully processed query on attempt 1
Successfully processed question 1

Processing question 2/5
Query: Which year did AnyCompany Financial have the highest net cash used in investing activities, what was...
Successfully processed query on attempt 1
Successfully processed question 2

Processing question 3/5
Query: What was the primary source of cash inflows from financing activities for AnyCompany Financial in 20...
Successfully processed query on attempt 1
Successfully processed question 3

Processing question 4/5
Query: Calculate the year-over-year percentage change in cash and cash equivalents for AnyCompany Financial...
Successfully processed query on attempt 1
Successfully processed question 4

Processing question 5/5
Query: With the information provided, what can you infer about AnyCompany Financial's overall 

6. 다음의 코드 셀을 실행하여 LLM의 답변과 평가 질문 세트에 대한 Ground Truth를 확인합니다.

In [7]:
i=0
for answer in answers:
    i=i+1
    print(str(i)+').'+questions[i-1]+'\n')
    print("LLM:" +answer+'\n')
    print ("Ground truth: "+ ground_truth[i-1]+'\n')

1).What was the primary reason for the increase in net cash by operating activities for AnyCompany Financial in 2021?

LLM:The primary reason for the increase in net cash provided by operating activities for AnyCompany Financial in 2021 was due to an increase in net income and favorable changes in operating assets and liabilities. Specifically, net cash provided by operating activities increased from $880 million in 2020 to $1,100 million in 2021, a $220 million increase. This rise was mainly attributed to an increase in net income from $1,125 million in 2020 to $1,200 million in 2021, and favorable changes in operating assets and liabilities.

Ground truth: An increase in net cash provided by operating activities was primarily due to increases in net income and favorable changes in operating assets and liabilities.

2).Which year did AnyCompany Financial have the highest net cash used in investing activities, what was the primary reason?

LLM:AnyCompany Financial had the highest net c

## 태스크 3.5: RAG 애플리케이션 평가

이 태스크에서는 *ragas.metrics*에서 사용할 모든 지표를 가져옵니다. 그런 다음, **evaluate()** 함수를 사용하여 관련 지표와 준비된 데이터세트를 전달하기만 하면 됩니다.

7. 다음의 코드 셀을 실행하여 **ragas.metrics**에서 모든 지표를 가져오고 **evaluate()** 함수를 사용합니다.

In [8]:
import warnings
import logging

warnings.filterwarnings('ignore')   # ignore warnings related to pydantic v1 to v2 migration
logging.getLogger('root').setLevel(logging.CRITICAL)

from datasets import Dataset
if not hasattr(Dataset, 'from_list'):
    def from_list_compatibility(data_list):
        if isinstance(data_list, list) and len(data_list) > 0 and isinstance(data_list[0], dict):
            keys = data_list[0].keys()
            data_dict = {key: [item[key] for item in data_list] for key in keys}
            return Dataset.from_dict(data_dict)
        return Dataset.from_dict({})
    Dataset.from_list = staticmethod(from_list_compatibility)

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_similarity,
    answer_correctness
)

from ragas.metrics.critique import (
harmfulness, 
maliciousness, 
coherence, 
correctness, 
conciseness
)

#specify the metrics here
metrics = [
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
        context_entity_recall,
        answer_similarity,
        answer_correctness,
        harmfulness, 
        maliciousness, 
        coherence, 
        correctness, 
        conciseness
    ]

try:
    result = evaluate(
        dataset=dataset,
        metrics=metrics,
        llm=llm_for_evaluation,
        embeddings=bedrock_embeddings,
    )
    df = result.to_pandas()
except Exception as e:
    # Handle any exceptions that occur during the evaluation
    print(f"An error occurred: {e}")

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

<i aria-hidden="true" class="fas fa-exclamation-circle" style="color:#7C5AED"></i> **주의:** 위 출력의 경고는 무시해도 됩니다. 계속 진행하기 전에 평가가 100% 완료되었는지 확인하십시오. 이 단계를 완료하는 데 약 7~10분이 소요됩니다.

8. 아래의 코드 셀을 실행하여 RAGAS 점수 결과를 확인합니다.

In [9]:
import pandas as pd
pd.options.display.max_colwidth = 10
df.style.set_sticky(axis="columns")

9. 아래의 코드 셀을 실행하여 결과 RAGAS 점수를 Excel 형식으로 내보냅니다.

In [10]:
df.style.to_excel('styled.xlsx', engine='openpyxl')

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 위 코드 셀을 성공적으로 실행하면 *styled.xlsx*라는 이름의 파일을 **en_us** 폴더의 왼쪽 탐색 창에서 검토할 수 있어야 합니다. 파일을 여는 데 시간이 너무 오래 걸리면 파일을 마우스 우클릭하고 **새 브라우저 탭에서 열기**를 선택합니다.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 위의 점수는 RAG 애플리케이션의 성능에 대한 상대적인 아이디어를 제공합니다. 따라서 주의해서 사용해야 하며 단독 점수로 사용해서는 안 됩니다. 또한 평가에는 5개의 문답 쌍만 사용했다는 점도 참고해야 합니다. 모델을 평가하려면 문서의 다양한 측면을 다루기에 충분한 데이터를 사용하는 것이 가장 좋습니다.

점수를 기준으로 RAG 워크플로의 다른 구성 요소를 검토하면 점수를 한층 더 최적화할 수 있습니다. 청킹 전략을 검토하고, 지침을 프롬프트에 입력하며, 추가 컨텍스트를 위해 numberOfResults를 추가할 수 있는 몇 가지 권장 옵션이 있습니다. 

<i aria-hidden="true" class="far fa-thumbs-up" style="color:#008296"></i> **태스크 완료:** 이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 태스크 4를 계속 진행합니다.